# 정적인 웹사이트, 동적인 웹사이트
_본 자료는 안수찬 강사님의 파이썬을 활용한 업무자동화 Camp (fast campus)의 강의자료를 기반으로 만들어졌습니다._  
만든이 : 김보섭  

## Preliminary
* 웹 사이트 (웹 클라이언트) : HTML, CSS, Javascript
* HTML : 사이트의 구조
* CSS : 사이트의 스타일
* Javascript : 사이트의 동적인 기능들 (애니메이션, 데이터를 서버에서 불러오는 기능)
* 데이터를 우리에게 뿌려주는 주체
    * 웹 서버 => HTML (Server rendering)
    * 웹 클라이언트 (Javascript) => HTML (Client Rendering)

## Summary
* __정적인 웹사이트__ _(requests.get HTML => BeautifulSoup CSS Selector)_
    * HTML 파일을 다운로드 (Crawling, Scraping)
    * 우리가 원하는 데이터의 위치를 찾아서 추출! (Parsing)
        * CSS Selector
  
  
* __동적인 웹사이트__ _(API URL (Headers) => requests.get JSON => Dict)_
    * 어딘가에서(\_____________:: API) 찾는 과정!
    * 데이터를 불러오고
    * 데이터를 추출하는 (parsing)

## HTTP Method (서버에 요청하는 방식)
* GET - URL을 통해서 데이터(요청)을 보낸다. (브라우져에서 서버로)
* POST - HTTP Body를 통해서 데이터(요청)을 보낸다.

## 정적인 웹사이트
### 네이버 실시간 검색어 데이터

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get('http://www.naver.com/')
response.status_code # 200이 뜨면 데이터를 잘 받아왔다는 뜻

200

In [3]:
response.text # HTML Text 중에서, 우리가 원하는 데이터 parsing!
type(response.text)

str

In [4]:
# HTML 구조를 계층적으로! (DOM : Document Object Model)
# HTML > BODY > DIV > UL > LI ...
#             > DIV > OL > LI
dom = BeautifulSoup(response.text, 'html.parser') 

In [5]:
rank_elements = dom.select('ol#realrank li.up')

In [6]:
rank_element = rank_elements[0]

In [7]:
rank_element

<li class="up" value="1"><a href="http://search.naver.com/search.naver?where=nexearch&amp;query=%EC%B1%84%EC%88%98%EB%B9%88&amp;sm=top_lve&amp;ie=utf8" title=""><span class="ell">채수빈</span><span class="tx">상승</span><span class="ic"></span><span class="rk">171</span></a></li>

In [8]:
rank_element.select_one('span.ell').text

'채수빈'

In [9]:
ranks = [rank_element.select_one('span.ell').text for rank_element in rank_elements ]
ranks

['채수빈',
 '역적',
 '에이미',
 '역적 ost',
 '이수민',
 '한국 중국',
 '프리스틴',
 '학교폭력 실태조사',
 '애플',
 '그녀는 거짓말을 너무 사랑해',
 '몬스타엑스',
 '박근혜',
 '외부자들',
 '하이라이트',
 'v앱',
 '이비에스아이',
 '아프리카tv',
 '프리즌']

### 네이버 블로그 포스트 가져오기
* 포스트 제목
* 포스트 주소  

In [10]:
response = requests.get('https://search.naver.com/search.naver?where=post&sm=tab_jum&ie=utf8&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC')

In [11]:
dom = BeautifulSoup(response.text, 'html.parser')

In [12]:
post_elements = dom.select('ul#elThumbnailResultArea li.sh_blog_top')

In [13]:
# <a href="http://blog.naver.com/__________/123/"></a>
# a (anchor) :: href (hyperlink reference)
data = [{'title' : post_element.select_one('a.sh_blog_title').attrs.get('title'),
         'url' : post_element.select_one('a.sh_blog_title').attrs.get('href')} for post_element in post_elements]
data

[{'title': '파이썬프로그래밍 기업에서 선호하는 이유', 'url': 'http://rmsep39.tistory.com/1905'},
 {'title': '초보자를 위한 파이썬 200제',
  'url': 'http://blog.naver.com/infopub?Redirect=Log&logNo=220945501558'},
 {'title': '파이썬 프로그래밍 배워야하는 3가지이유!', 'url': 'http://1984.tistory.com/448'},
 {'title': '마인크래프트를 활용한 파이썬 프로그래밍 과정 시작!',
  'url': 'http://chogar.blog.me/220942149662'},
 {'title': '프로그래밍 입문 파이썬부터 시작!!', 'url': 'http://edujoa.tistory.com/1389'},
 {'title': '오픈소스 언어로 만나는 데이터 분석, ‘파이썬’과 ‘R’',
  'url': 'http://blog.lgcns.com/1363'},
 {'title': '대구 프로그래밍 학원 파이썬 1개월 완성 주말 과정 개설',
  'url': 'http://blog.naver.com/nasu0210?Redirect=Log&logNo=220932224509'},
 {'title': '패치되지 않은 파이썬과 자바 취약점, 해커들이 FTP 인젝션을 통해 방화벽 우회하도록 허용해',
  'url': 'http://blog.alyac.co.kr/985'},
 {'title': '[개발] 파이썬 다운로드 및 윈도우에 설치하는 방법',
  'url': 'http://shaeod.tistory.com/949'},
 {'title': '파이썬 강좌, 비전공자도 배울 수 있다!', 'url': 'http://rmsep39.tistory.com/2056'}]

### 네이버 블로그 포스트 가져오기 (Page 넘어가면서)
HTTP method 중 GET 방식, URL에 데이터를 요청하므로, 아래의 example을 보면 URL parameter인 query에 특정값이 할당되어있음을 볼 수 있다.  
_(URL parameter인 query에 =로 써있는 값은 '파이썬')_  

example : https://search.naver.com/search.naver?where=post&sm=tab_jum&ie=utf8&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC  
example(page 정보포함) :  
https://search.naver.com/search.naver?where=post&sm=tab_pge&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&st=sim&date_option=0&dup_remove=1&srchby=all&ie=utf8&start=1
  
따라서 Page를 넘어가면서 네이버 블로그 '파이썬' 검색결과에서 블로그의 포스트의 제목과 URL을 가져오는 코드는 아래와 같이 구현할 수 있다.

In [14]:
base_url = 'https://search.naver.com/search.naver?where=post&sm=tab_pge&query=파이썬&start={page}'

In [15]:
data = []
for page in range(1,101,10):
    response = requests.get(base_url.format(page = page))
    dom = BeautifulSoup(response.text, 'html.parser')
    post_elements = dom.select('ul#elThumbnailResultArea li.sh_blog_top')
    data.append([{'title' : post_element.select_one('a.sh_blog_title').attrs.get('title'),
             'url' : post_element.select_one('a.sh_blog_title').attrs.get('href')} for post_element in post_elements])

In [16]:
# 과제
# 특정 파일(keywords.txt)을 읽어서
# 각각의 키워드에 해당하는 상위노출된 블로그 주소, 제목을 크롤링하고,
# 각각의 키워드명.csv 파일에다가 저장하기
import os
os.getcwd()

'D:\\dev\\py-automate'

## 동적인 웹사이트
* 실제 데이터가 어디서부터 오는지 체크 (예제 : 직방)
* 실제 웹 브라우저를 켜서 직방을 들어가는 방법 (Selenium) (다음 notebook 파일에)

### 직방 예제 (실제 주소찾기)
* 직방 사이트 :: 직방 개발자들 :: 직방 웹 클라이언트 : 직방 웹 서버 (API Server)  
* JSON (JSON API) :: Javascript Obejct Notation == Python Dict (매우 유사!)  
* requests.get => str(JSON Format String)

#### Python Dict => JSON Format String
json.dumps()

In [17]:
# Python Dict ==> JSON Format String
import json
student = {'name' : 'Boseop Kim', 'email' : 'svei89@korea.ac.kr'}
json_text = json.dumps(student)

In [18]:
print(json_text, type(json_text))

{"name": "Boseop Kim", "email": "svei89@korea.ac.kr"} <class 'str'>


####  JSON Format String => Python Dict
json.loads()

In [19]:
json_text

'{"name": "Boseop Kim", "email": "svei89@korea.ac.kr"}'

In [20]:
json.loads(json_text)

{'email': 'svei89@korea.ac.kr', 'name': 'Boseop Kim'}

In [21]:
type(json.loads(json_text))

dict

#### 직방 예제
실제 매물정보가 담긴 URL을 크롬 개발자 도구를 이용하여 찾아서 진행, string을 받아왔을 때, 직방의 매물이 json의 형태로 담겨있으므로 바로 json 패키지를 이용하여 진행  

#### 보증금과 월세 가져오기

In [22]:
response = requests.get('https://api.zigbang.com/v3/items?detail=true&item_ids=[7468696,7780221,7747863,7699078,7250207,7779822,7593985,7672971,7557867,7662179,7728008,7748278,7590167,7753835,7684547,7767467,7764519,7703739,7729634,7792521,7747993,7736951,7515215,7577402,7720787,7603913,7721071,7755858,7775742,7755021,7694602,7566668,7722769,7791292,6915486,7542897,7618794,7772538,7690527,7777154,7745980,7710823,7685570,7780189,7775527,7770996,7782847,7750303,7715553,7493782,7736339,7663573,7645807,7652183,7439872,7768911,7584389,7615213,7718616,7720892]')

In [23]:
zigbang = json.loads(response.text)
# zigbang = response.json() 위와 동일한 기능 

In [24]:
type(zigbang)

dict

In [25]:
zigbang['items'][0]['item']['rent']

68

In [26]:
zigbang['items'][0]['item']['deposit']

1000

#### 전체 매물에서 월세, 보증금을 뽑기
dict의 List 형태 (deposit, rent)

In [27]:
#for item in zigbang.get('items'):
#    deposit = item.get('item').get('deposit')
#    rent = item.get('item').get('rent')
#    print(deposit, rent)
[{'deposit': item.get('item').get('deposit'),
  'rent': item.get('item').get('rent')} for item in zigbang.get('items')]

[{'deposit': 1000, 'rent': 68},
 {'deposit': 2000, 'rent': 75},
 {'deposit': 2000, 'rent': 30},
 {'deposit': 500, 'rent': 55},
 {'deposit': 1000, 'rent': 60},
 {'deposit': 15000, 'rent': 0},
 {'deposit': 1000, 'rent': 65},
 {'deposit': 95, 'rent': 95},
 {'deposit': 25000, 'rent': 0},
 {'deposit': 3000, 'rent': 30},
 {'deposit': 2000, 'rent': 65},
 {'deposit': 1000, 'rent': 60},
 {'deposit': 25000, 'rent': 0},
 {'deposit': 500, 'rent': 50},
 {'deposit': 5000, 'rent': 65},
 {'deposit': 22000, 'rent': 0},
 {'deposit': 2000, 'rent': 80},
 {'deposit': 14000, 'rent': 0},
 {'deposit': 1000, 'rent': 45},
 {'deposit': 1000, 'rent': 65},
 {'deposit': 500, 'rent': 60},
 {'deposit': 1000, 'rent': 75},
 {'deposit': 500, 'rent': 55},
 {'deposit': 105, 'rent': 105},
 {'deposit': 100, 'rent': 58},
 {'deposit': 5000, 'rent': 50},
 {'deposit': 2000, 'rent': 60},
 {'deposit': 20000, 'rent': 0},
 {'deposit': 500, 'rent': 65},
 {'deposit': 17000, 'rent': 0},
 {'deposit': 1000, 'rent': 38},
 {'deposit': 100

### 요기요 예제 (개발자 도구에서 response에서는 보이나 해당 URL을 쳐서들어가면 안보이는 경우)
아래의 링크의 식당 이름 20개 가져오기  
링크 : https://www.yogiyo.co.kr/mobile/#/%EC%84%9C%EC%9A%B8/139231/

* Referer : Request가 어디서부터 왔는가?
    * 개발자 도구 : yogiyo.co.kr/
    * 직접 URL 입력 : Referer가 없는 상태  
    
    
* Request Header (http://docs.python-requests.org/en/master/user/quickstart/)
    * Authorization
    * X---
    * Host

In [28]:
url = 'https://www.yogiyo.co.kr/api/v1/restaurants-geo/?items=20&order=rank&page=0&search=&zip_code=139231'
# 위에 url query에 items를 바꿔서 한번에 여러개 긁어오기도 가능 (API가 잘 구성된 사이트라면!)
headers = {'X-ApiKey' : 'iphoneap',
           'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2'}
response = requests.get(url, headers = headers)

In [29]:
yogiyo = response.json()
[restaurant.get('name') for restaurant in yogiyo.get('restaurants')]

['7번가피자-상계점',
 'BHC-하계점',
 'BHC-하계점',
 '후라이드참잘하는집-중계점',
 'BHC-중계점',
 '치킨레인저-노원기지',
 '앵그리불닭발',
 '땡초불닭발동대문엽기떡볶이-월계점',
 '빨간고추피자-공릉점',
 '83닭발-노원점',
 '엄청난쌀국수-본점',
 "강'스피자-since2000",
 '정성커리&돈카츠-노원점',
 '바다회포차',
 '분식명인김라덕선생-노원점',
 '피자마루-공릉점',
 '강정구의피자생각-상계점',
 '호식이두마리치킨-월계1호점',
 '왕족발칼국수',
 '마왕족발-노원점']